In [1]:
import sys

In [2]:
sys.path.insert(0, "..")

In [3]:
import spacy
import medspacy
from medspacy.visualization import visualize_ent, visualize_dep

# Overview
In this notebook, we'll look at how to extract clinical concepts and attributes from text.
- Target matching
- Context analysis

In [4]:
with open("./discharge_summary.txt") as f:
    text = f.read()

In [5]:
nlp = medspacy.load(enable=["pyrush"])

# Target extraction
In this step, we'll write rules to extract the main concepts we're interested in.

In this example, we'll use two utilities provided in `medspacy.ner` for rule-based matching: the `TargetMatcher` and `TargetRule`. However, you can use any spaCy components for adding spans to `doc.ents`, including pre-trained NER models or other [spaCy rule-based matching components](https://spacy.io/usage/rule-based-matching/).

If you want to use a statistical NER model instead of a rule-based component, see [./6-Using-Pretrained-Models.ipynb](./6-Using-Pretrained-Models.ipynb)

## Target concepts
In our text, we'll extract the following concepts:
- Diagnoses 
- Medications

In addition, we'll show a few examples of how to add a custom spaCy attribute to a target rule to add an ICD-10 diagnosis code as an attribute of an entity.

First, we'll import `TargetMatcher` and `TargetRule`. We'll instantiate a `TargetRule` and add it to our pipeline.

In [6]:
from medspacy.ner import TargetMatcher, TargetRule

In [7]:
target_matcher = TargetMatcher(nlp)

In [8]:
target_matcher = nlp.add_pipe("medspacy_target_matcher")

Now we will define some rules for extract concepts from the text. The `TargetRule` class defines rules for extracting entities from the text. `TargetRule` takes the following arguments:
- `literal`: An exact phrase to match in the text
- `category`: The semantic class of the entity (ie., `ent.label_`)
- `pattern`: An optional pattern to match rather than `literal`. As we'll see below, this can be either a list of dictionaries defining token attributes or a regular expression string
- `on_match`: An optional callback function. See https://spacy.io/usage/rule-based-matching#on_match
- `metadata`: An optional dictionary of metadata
- `attributes`: An optional dictionary of custom attributes to set in the resulting entity. We'll see an example below.

Let's define a few simple rules:

In [9]:
target_rules1 = [
    TargetRule(literal="abdominal pain", category="PROBLEM"),
    TargetRule("stroke", "PROBLEM"),
    TargetRule("hemicolectomy", "TREATMENT"),
    TargetRule("Hydrochlorothiazide", "TREATMENT"),
    TargetRule("colon cancer", "PROBLEM"),
    TargetRule("metastasis", "PROBLEM"),
    
]

In [10]:
target_matcher.add(target_rules1)

In [11]:
doc = nlp(text)

In [12]:
print(doc.ents)

(Hydrochlorothiazide, Abdominal pain, stroke, abdominal pain, metastasis, Colon cancer, hemicolectomy, stroke, abdominal pain, abdominal pain)


In [13]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.target_rule.literal, sep="  |  ")
    print()

Hydrochlorothiazide  |  TREATMENT  |  Hydrochlorothiazide

Abdominal pain  |  PROBLEM  |  abdominal pain

stroke  |  PROBLEM  |  stroke

abdominal pain  |  PROBLEM  |  abdominal pain

metastasis  |  PROBLEM  |  metastasis

Colon cancer  |  PROBLEM  |  colon cancer

hemicolectomy  |  TREATMENT  |  hemicolectomy

stroke  |  PROBLEM  |  stroke

abdominal pain  |  PROBLEM  |  abdominal pain

abdominal pain  |  PROBLEM  |  abdominal pain



## Advanced Pattern Matching
SpaCy has powerful pattern matching which allows you to match on a list of dictionaries which define attributes for each token. See https://spacy.io/usage/rule-based-matching for spaCy's documentation and examples. Additionally, medspaCy allows matching with regular expressions on the underlying text of the doc.

Let's see some examples of `TargetRules` which utilizie pattern matching. The first rule uses token patterns to match any single token where the lower-case text is either `xrt` or `radiotherapy`. The second uses regular expressions to match various forms of Type 1/Type 2 diabetes.

In [14]:
pattern_rules = [
    TargetRule("radiotherapy", "PROBLEM",
              pattern=[{"LOWER": {"IN": ["xrt", "radiotherapy"]}}]
              ),
    TargetRule("diabetes", "PROBLEM",
              pattern=r"type (i|ii|1|2|one|two) (dm|diabetes mellitus)"),
                ]

In [15]:
target_matcher.add(pattern_rules)

../medspacy/common/regex_matcher.py:60: RuntimeWarning: You are using a TargetRule with a regex pattern, which is not natively supported in spacy and may lead to unexpected match spans. Consider using a list of dicts pattern instead. See https://spacy.io/usage/rule-based-matching
  warnings.warn(


In [16]:
doc = nlp(text)

We can now see several new entities have been added to our entities: "XRT", "type 2 dm", "Type II Diabetes Mellitus", "Type 2 DM".

In [17]:
print(doc.ents)

(Hydrochlorothiazide, Abdominal pain, type 2 dm, stroke, abdominal pain, metastasis, Colon cancer, hemicolectomy, XRT, Type II Diabetes Mellitus, stroke, Type 2 DM, abdominal pain, abdominal pain)


### A note about regular expressions
Regular-expression matching is not natively supported by spaCy and could result in unexpected matched spans if match boundaries do not align with token boundaries. For example, let's say you have a pattern like this one which matches a span of text which occurs between 2 different tokens:

In [18]:
import re

example_text = "SERVICE: Radiology"
pattern_str = r"ICE: Rad"

In this case, medspaCy will find the closest token boundaries around the matched span, meaning that the resulting `ent` won't actually equal the matched span of text. Because of this, it is recommended to use a list of dicts whenever possible.

In [19]:
target_matcher.add([TargetRule("ICE: Rad", "SERVICE", pattern=pattern_str)])

../medspacy/common/regex_matcher.py:60: RuntimeWarning: You are using a TargetRule with a regex pattern, which is not natively supported in spacy and may lead to unexpected match spans. Consider using a list of dicts pattern instead. See https://spacy.io/usage/rule-based-matching
  warnings.warn(


In [20]:
nlp(example_text).ents

(SERVICE: Radiology,)

In [21]:
re.search(pattern_str, example_text)

<re.Match object; span=(4, 12), match='ICE: Rad'>

## Adding custom attributes
One of the most powerful functionalities of spaCy is the ability to add [custom attributes](https://spacy.io/usage/processing-pipelines#custom-components-attributes) to spaCy objects (`Doc`, `Span`, or `Token`). These custom attributes are accessed through `obj._....`. As we'll see in later steps, medSpaCy adds several of these attributes in other attributes like `context` or `sectionizer`. 

But it is sometimes useful to include a custom attribute as part of the target matching rule, rather than needing to build a separate component to add it. The `TargetRule` can also include a value for these attributes in the `attributes` argument. 

For example, let's say we want to map certain entities to [ICD-10 diagnosis codes](https://www.cdc.gov/nchs/icd/icd10cm.htm). One way we can do this is to include the diagnosis codes for concepts in our knowledge base. For example, **"Type II Diabetes"** can be mapped to **E11.9**". We can add this to entities by first registering the extension for the `Span` class:


In [22]:
from spacy.tokens import Span

In [23]:
Span.set_extension("icd10", default="")

We can now include ICD-10 code values in the target rules. We'll map **"Type II Diabetes Mellitus"** to **"E11.9"** and **"Hypertension"** to **"I10"**:

In [24]:
target_rules2 = [
    TargetRule("Type II Diabetes Mellitus", "PROBLEM", 
              pattern=[
                  {"LOWER": "type"},
                  {"LOWER": {"IN": ["2", "ii", "two"]}},
                  {"LOWER": {"IN": ["dm", "diabetes"]}},
                  {"LOWER": "mellitus", "OP": "?"}
              ],
              attributes={"icd10": "E11.9"}),
    TargetRule("Hypertension", "PROBLEM",
              pattern=[{"LOWER": {"IN": ["htn", "hypertension"]}}],
              attributes={"icd10": "I10"}),
    
    
]

In [25]:
target_matcher.add(target_rules2)

In [26]:
doc = nlp(text)

Now, whenever one of these rules results in a match, the ICD-10 value can be accessed in `ent._.icd10`:

In [27]:
for ent in doc.ents:
    if ent._.icd10 != "":
        print(ent, ent._.icd10)

type 2 dm E11.9
Type II Diabetes Mellitus E11.9
Hypertension I10
Type 2 DM E11.9
HTN I10


# Context
Clinical text often contains mentions of concepts which the patient did not actually experience. For example:

- "There is *no evidence of* **pneumonia**"
- "*Mother* with **breast cancer**"
- "Patient presents for *r/o* **COVID-19**"

In all of these instances, we need to use the contextual clues around the entity to assert attributes like negation, experiencer, and uncertainty.

One method for this is the [ConText algorithm](https://www.sciencedirect.com/science/article/pii/S1532046409000744). ConText links target entities like problems with semantic modifiers like those shown above. The medSpaCy implementation of ConText is found in `medspacy.context`.

The ConText component is explained in more detail in the `context/` notebooks.

In [28]:
from medspacy.context import ConTextComponent, ConTextRule

In [29]:
context = ConTextComponent(nlp, rules="default")

In [30]:
context = nlp.add_pipe("medspacy_context", config={"rules": "default"})

In [31]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

In [32]:
doc = nlp("Mother with stroke at age 82.")

We can use medSpaCy visualizers from the module `medspacy.visualization` to show the target/modifiers in text. `visualize_dep` shows arrows targets to show which concepts are modified by the semantic modifiers:

In [33]:
visualize_ent(doc)

In [34]:
visualize_dep(doc)

In [35]:
short_doc = nlp("Colon cancer diagnosed in 2012")

We can add a new rule using the `ConTextRule` class, which behaves similarly to `TargetRule` with some additional arguments defining modifier behavior:

In [36]:
context_rules = [
    ConTextRule("diagnosed in <YEAR>", "HISTORICAL", 
                rule="BACKWARD", # Look "backwards" in the text (to the left)
               pattern=[
                   {"LOWER": "diagnosed"},
                   {"LOWER": "in"},
                   {"LOWER": {"REGEX": "^[\d]{4}$"}}
               ])
]

<>:7: DeprecationWarning: invalid escape sequence \d
<>:7: DeprecationWarning: invalid escape sequence \d
<ipython-input-36-f01b1240e78d>:7: DeprecationWarning: invalid escape sequence \d
  {"LOWER": {"REGEX": "^[\d]{4}$"}}
../medspacy/context/context_rule.py:136: DeprecationWarning: The 'rule' argument from ConTextItem has been replaced with 'direction' in ConTextRule. In the future please use 'direction': ConTextItem(literal, category, direction=...)
  warnings.warn("The 'rule' argument from ConTextItem has been replaced with 'direction' "


In [37]:
context.add(context_rules)

In [38]:
short_doc = nlp("Colon cancer diagnosed in 2012")

In [39]:
visualize_ent(short_doc)

In [40]:
visualize_dep(short_doc)

In addition to linking targets and modifiers, ConText will also set attributes for each entity:

In [41]:
for ent in doc.ents:
    if any([ent._.is_negated, ent._.is_uncertain, ent._.is_historical, ent._.is_family, ent._.is_hypothetical, ]):
        print("'{0}' modified by {1} in: '{2}'".format(ent, ent._.modifiers, ent.sent))
        print()

'stroke' modified by (<ConTextModifier> [Mother, FAMILY],) in: 'Mother with stroke at age 82.'

